# DataFrames: Reading in messy data
     
In the [01-data-access](./01-data-access.ipynb) example we show how Dask Dataframes can read and store data in many of the same formats as Pandas dataframes.  One key difference, when using Dask Dataframes is that instead of opening a single file with a function like [pandas.read_csv](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html), we typically open many files at once with [dask.dataframe.read_csv](https://docs.dask.org/en/latest/dataframe-api.html#dask.dataframe.read_csv). This enables us to treat a collection of files as a single dataset. Most of the time this works really well. But real data is messy and in this notebook we will explore a more advanced technique to bring messy datasets into a dask dataframe.

## Start Dask Client for Dashboard

Starting the Dask Client is optional.  It will provide a dashboard which 
is useful to gain insight on the computation.  

The link to the dashboard will become visible when you create the client below.  We recommend having it open on one side of your screen while using your notebook on the other side.  This can take some effort to arrange your windows, but seeing them both at the same is very useful when learning.

In [1]:
from dask.distributed import Client
client = Client(n_workers=1, threads_per_worker=4, processes=False, memory_limit='2GB')
client

Client Scheduler: inproc://10.20.1.11/13446/1 Dashboard: http://10.20.1.11:8787/status,Cluster Workers: 1 Cores: 4 Memory: 2.00 GB


## Create artificial dataset

First we create an artificial dataset and write it to many CSV files.

You don't need to understand this section, we're just creating a dataset for the rest of the notebook.

In [2]:
import dask
df = dask.datasets.timeseries()
df

,id,name,x,y
npartitions=30,,,,
2000-01-01,int64,object,float64,float64
2000-01-02,...,...,...,...
...,...,...,...,...
2000-01-30,...,...,...,...
2000-01-31,...,...,...,...


In [3]:
import os
import datetime

if not os.path.exists('data'):
    os.mkdir('data')

def name(i):
    """ Provide date for filename given index
    
    Examples
    --------
    >>> name(0)
    '2000-01-01'
    >>> name(10)
    '2000-01-11'
    """
    return str(datetime.date(2000, 1, 1) + i * datetime.timedelta(days=1))
    
df.to_csv('data/*.csv', name_function=name, index=False);

## Read CSV files

We now have many CSV files in our data directory, one for each day in the month of January 2000.  Each CSV file holds timeseries data for that day.  We can read all of them as one logical dataframe using the `dd.read_csv` function with a glob string.

In [4]:
!ls data/*.csv | head

data/2000-01-01.csv
data/2000-01-02.csv
data/2000-01-03.csv
data/2000-01-04.csv
data/2000-01-05.csv
data/2000-01-06.csv
data/2000-01-07.csv
data/2000-01-08.csv
data/2000-01-09.csv
data/2000-01-10.csv


In [5]:
import dask.dataframe as dd

df = dd.read_csv('data/2000-*-*.csv')
df

,id,name,x,y
npartitions=30,,,,
,int64,object,float64,float64
,...,...,...,...
...,...,...,...,...
,...,...,...,...
,...,...,...,...


In [6]:
df.head()

,id,name,x,y
0,952,Norbert,0.401557,0.216120
1,1014,Victor,0.517099,-0.605371
2,1027,Victor,0.924862,0.153773
3,1034,Jerry,0.928455,-0.544399
4,1084,Sarah,0.144669,-0.985843


Let's look at some statistics on the data

In [7]:
df.describe().compute()

,id,x,y
count,2.592000e+06,2.592000e+06,2.592000e+06
mean,1.000010e+03,6.968149e-04,-1.744684e-04
std,3.163116e+01,5.772712e-01,5.773211e-01
min,8.530000e+02,-9.999994e-01,-9.999989e-01
25%,9.790000e+02,-4.934138e-01,-4.941785e-01
50%,1.000000e+03,7.721299e-03,5.889143e-03
75%,1.022000e+03,5.072985e-01,5.045190e-01
max,1.156000e+03,9.999991e-01,9.999990e-01


# Make some messy data

Now this works great, and in most cases dd.read_csv or dd.read_parquet etc are the preferred way to read in large collections of data files into a dask dataframe, but real world data is often very messy and some files may be broken or badly formatted. To simulate this we are going to create some fake messy data by tweaking our example csv files. For the file `data/2000-01-05.csv` we will replace with no data and for the file `data/2000-01-07.csv` we will remove the `y` column 

In [8]:
# corrupt the data in data/2000-01-05.csv
with open('data/2000-01-05.csv', 'w') as f:
    f.write('')

In [9]:
# remove y column from data/2000-01-07.csv
import pandas as pd
df = pd.read_csv('data/2000-01-07.csv')
del df['y']
df.to_csv('data/2000-01-07.csv', index=False)

In [10]:
!head data/2000-01-05.csv

In [11]:
!head data/2000-01-07.csv

id,name,x
959,Michael,0.3799552732457041
1017,Norbert,0.374581500157352
959,Ingrid,0.9463408655558982
976,Yvonne,0.5037412239572392
1081,Bob,-0.7974129240981789
1050,Charlie,-0.3138982789715463
1021,Kevin,-0.12748721803911467
1057,Laura,-0.6274480655807837
996,Michael,-0.023838029000380345


# Reading the messy data

Let's try reading in the collection of files again

In [12]:
df = dd.read_csv('data/2000-*-*.csv')

In [13]:
df.head()

,id,name,x,y
0,952,Norbert,0.401557,0.216120
1,1014,Victor,0.517099,-0.605371
2,1027,Victor,0.924862,0.153773
3,1034,Jerry,0.928455,-0.544399
4,1084,Sarah,0.144669,-0.985843


Ok this looks like it worked, let us calculate the dataset statistics again

In [14]:
df.describe().compute()

distributed.worker - WARNING -  Compute Failed
Function:  execute_task
args:      ((subgraph_callable, (<function pandas_read_text at 0x7fa436da94c0>, <function _make_parser_function.<locals>.parser_f at 0x7fa43ce9c820>, (<function read_block_from_file at 0x7fa4345821f0>, <OpenFile '/home/travis/build/dask/dask-examples/dataframes/data/2000-01-07.csv'>, 0, 64000000, b'\n'), b'id,name,x,y\n', {}, {'id': dtype('int64'), 'name': dtype('O'), 'x': dtype('float64'), 'y': dtype('float64')}, ['id', 'name', 'x', 'y'], False, False, None), 'read-csv-f82d51fc5a7458c9828be8fece21dc6c'))
kwargs:    {}
Exception: ValueError('Length mismatch: Expected axis has 3 elements, new values have 4 elements')



ValueError: Length mismatch: Expected axis has 3 elements, new values have 4 elements

So what happened? 

When creating a dask dataframe from a collection of files, dd.read_csv samples the first few files in the dataset to determine the datatypes and columns available. Since it has not opened all the files it does not now if some of them are corrupt. Hence, `df.head()` works since it is only looking at the first file. `df.describe.compute()` fails because of the corrupt data in `data/2000-01-05.csv`

# Building a delayed reader

To get around this problem we are going to use a more advanced technique to build our dask dataframe. This method can also be used any time some custom logic is required when reading each file. Essentially, we are going to build a function that uses  pandas and some error checking and returns a pandas dataframe. If we find a bad data file we will either find a way to fix/clean the data or we will return and empty pandas dataframe with the same structure as the good data.

In [15]:
import numpy as np
import io

def read_data(filename):
    
    # for this to work we need to explicitly set the datatypes of our pandas dataframe 
    dtypes = {'id': int, 'name': str, 'x': float, 'y': float}
    try:
        # try reading in the data with pandas 
        df = pd.read_csv(filename, dtype=dtypes)
    except:
        # if this fails create an empty pandas dataframe with the same dtypes as the good data
        df = pd.read_csv(io.StringIO(''), names=dtypes.keys(), dtype=dtypes)
    
    # for the case with the missing column, add a column of data with NaN's
    if 'y' not in df.columns:
        df['y'] = np.NaN
        
    return df

Let's test this function on a good file and the two bad files

In [16]:
# test function on a normal file
read_data('data/2000-01-01.csv').head()

,id,name,x,y
0,952,Norbert,0.401557,0.216120
1,1014,Victor,0.517099,-0.605371
2,1027,Victor,0.924862,0.153773
3,1034,Jerry,0.928455,-0.544399
4,1084,Sarah,0.144669,-0.985843


In [17]:
# test function on the empty file
read_data('data/2000-01-05.csv').head()

,id,name,x,y


In [18]:
# test function on the file missing the y column
read_data('data/2000-01-07.csv').head()

,id,name,x,y
0,959,Michael,0.379955,NaN
1,1017,Norbert,0.374582,NaN
2,959,Ingrid,0.946341,NaN
3,976,Yvonne,0.503741,NaN
4,1081,Bob,-0.797413,NaN


# Assembling the dask dataframe

First we take our `read_data` function and convert it to a dask delayed function

In [19]:
from dask import delayed
read_data = delayed(read_data)

Let us look at what the function does now

In [20]:
df = read_data('data/2000-01-01.csv')
df

Delayed('read_data-f91ae336-e20f-42dc-9376-fe19dd3f31f1')

It creates a delayed object, to actually run read the file we need to run `.compute()`

In [21]:
df.compute()

,id,name,x,y
0,952,Norbert,0.401557,0.216120
1,1014,Victor,0.517099,-0.605371
2,1027,Victor,0.924862,0.153773
3,1034,Jerry,0.928455,-0.544399
4,1084,Sarah,0.144669,-0.985843
...,...,...,...,...
86395,1023,Dan,-0.543266,-0.961339
86396,990,Norbert,0.823865,0.303777
86397,1039,Kevin,-0.872827,0.044623
86398,975,Xavier,0.353160,-0.869766


Now let's build a list of all the available csv files

In [22]:
# loop over all the files
from glob import glob
files = glob('data/2000-*-*.csv')
files

['data/2000-01-13.csv',
 'data/2000-01-02.csv',
 'data/2000-01-24.csv',
 'data/2000-01-04.csv',
 'data/2000-01-09.csv',
 'data/2000-01-17.csv',
 'data/2000-01-06.csv',
 'data/2000-01-08.csv',
 'data/2000-01-11.csv',
 'data/2000-01-28.csv',
 'data/2000-01-29.csv',
 'data/2000-01-18.csv',
 'data/2000-01-16.csv',
 'data/2000-01-05.csv',
 'data/2000-01-30.csv',
 'data/2000-01-20.csv',
 'data/2000-01-19.csv',
 'data/2000-01-21.csv',
 'data/2000-01-27.csv',
 'data/2000-01-12.csv',
 'data/2000-01-01.csv',
 'data/2000-01-07.csv',
 'data/2000-01-10.csv',
 'data/2000-01-25.csv',
 'data/2000-01-14.csv',
 'data/2000-01-26.csv',
 'data/2000-01-23.csv',
 'data/2000-01-03.csv',
 'data/2000-01-15.csv',
 'data/2000-01-22.csv']

Now we run the delayed read_data function on each file in the list

In [23]:
df = [read_data(file) for file in files]
df

[Delayed('read_data-838ff163-6fc4-4dc5-bd3d-d15a923be641'),
 Delayed('read_data-fd4d20ff-b33c-4ee1-b746-04099280a662'),
 Delayed('read_data-9c1f4408-c786-47e7-841e-dcc36df49d75'),
 Delayed('read_data-d88d9be0-1ddd-4a82-8a50-8002e7c02143'),
 Delayed('read_data-ff00a684-e51c-45e6-a771-233a65ced4b6'),
 Delayed('read_data-de7aa038-0e3a-487a-afad-771cc5fc85c4'),
 Delayed('read_data-738f180e-c856-4bb8-9154-c03b709f4bdb'),
 Delayed('read_data-50f28295-e95a-4d90-a5cc-b7dc7223a97c'),
 Delayed('read_data-c0738c34-7aa6-4289-99a9-3d12f4655c01'),
 Delayed('read_data-2cacab10-52ec-421a-817c-4cf76d8368f2'),
 Delayed('read_data-b0386671-b2ce-4742-a509-d21cef69ac87'),
 Delayed('read_data-b8cef8d3-dbcf-4c21-9c2a-971e46956d7e'),
 Delayed('read_data-929bdef4-e3cd-4204-86fb-36fa4966a4c7'),
 Delayed('read_data-93c9a7a4-cebc-4bc9-a45e-d67f9771c905'),
 Delayed('read_data-4b8aaf08-e508-4442-bb8f-b3d30cbb4c5a'),
 Delayed('read_data-33e26af9-4955-4d15-8026-82607ec1a28d'),
 Delayed('read_data-7a5a78f5-0acc-4846-8

Then we use [dask.dataframe.from_delayed](https://docs.dask.org/en/latest/dataframe-api.html#dask.dataframe.from_delayed). This function creates a Dask DataFrame from a list of delayed objects as long as each delayed object returns a pandas dataframe. The structure of each individual dataframe returned must also be the same.

In [24]:
df = dd.from_delayed(df, meta={'id': int, 'name': str, 'x': float, 'y': float})
df

,id,name,x,y
npartitions=30,,,,
,int64,object,float64,float64
,...,...,...,...
...,...,...,...,...
,...,...,...,...
,...,...,...,...


Note: we provided the dtypes in the `meta` keyword to explicitly tell Dask Dataframe what kind of dataframe to expect. If we did not do this Dask would infer this from the first delayed object which could be slow if it was a large csv file

## Now let's see if this works

In [25]:
df.head()

,id,name,x,y
0,1015,Michael,0.946620,-0.298323
1,1045,Laura,0.629723,0.964831
2,1001,Edith,-0.592952,0.074773
3,965,Ingrid,0.102165,-0.249787
4,1008,Wendy,-0.795261,0.681271


In [26]:
df.describe().compute()

/home/travis/miniconda/envs/test/lib/python3.8/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


,id,x,y
count,2.505600e+06,2.505600e+06,2.419200e+06
mean,1.000005e+03,6.388805e-04,-2.064705e-04
std,3.163227e+01,5.772376e-01,5.773267e-01
min,8.530000e+02,-9.999994e-01,-9.999989e-01
25%,9.790000e+02,-4.934138e-01,-4.941785e-01
50%,1.000000e+03,7.721299e-03,5.889143e-03
75%,1.022000e+03,5.072985e-01,5.045190e-01
max,1.156000e+03,9.999990e-01,9.999990e-01


## Success!

To recap, in this example, we looked at an approach to create a Dask Dataframe from a collection of many data files. Typically you would use built-in functions like `dd.read_csv` or `dd.read_parquet` to do this. Sometimes, this is not possible because of messy/corrupted files in your dataset or some custom processing that might need to be done. 

In these cases, you can build a Dask DataFrame with the following steps.

1. Create a regular python function that reads the data, performs any transformations, error checking etc and always returns a  Pandas dataframe with the same structure
2. Convert this read function to a delayed object using the `dask.delayed` function
3. Call each file in your dataset with the delayed data reader and assemble the output as a list of delayed objects
4. Used `dd.from_delayed` to covert the list of delayed objects to a Dask Dataframe 

This same technique can be used in other situations as well. Another example might be data files that require using a specialized reader, or several transformations before they can be converted to a pandas dataframe.